###### Assignment -02
##### Name: Vakeesan.K
##### Index NO: 190643G

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy import linalg
import matplotlib.pyplot as plt
import math
import random


# np.random.seed ( 0 )
N = 100
half_n = N// 2
r = 10
s = r /16
t = np.random.uniform(0 , 2*np.pi , half_n )
n = s*np.random.randn ( half_n )
x , y = ( r + n)*np.cos ( t ) , ( r + n)*np.sin ( t )
X_circ = np.hstack ( ( x.reshape ( half_n , 1 ) , y.reshape ( half_n , 1 ) ) )
m, b = -1, 2
x = np.linspace (-12, 12 , half_n )
y = m*x + b + s*np.random.randn ( half_n )
X_line = np.hstack ( ( x.reshape ( half_n , 1 ) , y.reshape ( half_n , 1 ) ) )
X = np.vstack ( ( X_circ , X_line ) )

#plot the coordinate set
fig, ax = plt.subplots()
xs = [x[0] for x in X]
ys = [x[1] for x in X]
plt.scatter(xs, ys)
plt.xlim(-20,20)
plt.ylim(-20,20)
#ransac
#our data set X={(xi,yi)}
num_iterations=math.inf
iterations_done =0

max_inlier_count = 0
best_model = None

prob_outlier = 0.5
desired_prob = 0.95
#temp_arr=[]
inlier_max_count=0
p=0.99 #ideal value
t=2 #ideal expected noise or threshold

while(num_iterations > iterations_done):
    #estimate x,y,r from three radom points
    cor_1,cor_2,cor_3= random.choices(X,k=3)
    #fina a circle through the three points
    x1=cor_1[0]
    y1=cor_1[1]
    x2=cor_2[0]
    y2=cor_2[1]
    x3=cor_3[0]
    y3=cor_3[1]

    c = (x1-x2)**2 + (y1-y2)**2
    a = (x2-x3)**2 + (y2-y3)**2
    b = (x3-x1)**2 + (y3-y1)**2
    s = 2*(a*b + b*c + c*a) - (a*a + b*b + c*c) 
    px = (a*(b+c-a)*x1 + b*(c+a-b)*x2 + c*(a+b-c)*x3) / s
    py = (a*(b+c-a)*y1 + b*(c+a-b)*y2 + c*(a+b-c)*y3) / s 
    ar = a**0.5
    br = b**0.5
    cr = c**0.5 
    r = ar*br*cr / ((ar+br+cr)*(-ar+br+cr)*(ar-br+cr)*(ar+br-cr))**0.5
    #select the points within the donut region
    
    inlier_count=0
    abs_error=0
    
    for i in X:
        x_=i[0]
        y_=i[1]
        if(abs(np.sqrt(pow(x_-px,2)+pow(y_-py,2))-r)<t):
            inlier_count+=1
            abs_error+=np.abs(np.sqrt(pow(x_-px,2)+pow(y_-py,2))-r)
            #temp_arr.append([x_,y_])

    #selecting the second circle 
    if (inlier_count>threshold_inlier_count):
        if ((inlier_max_count<=inlier_count) and inlier_count!=0):
            inlier_max_count=inlier_count
            arr.append([inlier_max_count,abs_error/inlier_count,px,py,r,cor_1,cor_2,cor_3]) 
    #loop condition
    prob_outlier = 1 - inlier_count/N
    try:
        num_iterations = math.log(1 - desired_prob)/math.log(1 - (1 - prob_outlier)**3)
    except ZeroDivisionError :
        pass
    iterations_done = iterations_done + 1
            
    
if(len(arr)>0):
    new_arr=[]
    max=arr[-1][0]
    for j in arr:
        if max<=j[0]:
            new_arr.append(j)
    #if there are so many points then look into abs mean error
    min_error=new_arr[0][1]
    index=0
    for k in range(len(new_arr)):
        if new_arr[k][1]<min_error:
            index=k
    cent_x=new_arr[index][2]
    cent_y=new_arr[index][3]
    cent_r=new_arr[index][4]
    print(cent_x,cent_y,cent_r)
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import random

def data_generator():
    x_data = []
    y_data = []

# 	a = 2
# 	b = 3
# 	r = 100

# 	for i in np.arange(a - r, a + r, 1):
# 		x = i
# 		y = 0
# 		y1, y2 = circle_func(a, b, r, x)
# 		if np.random.randn() > 0:
# 			y += y1 + np.random.randn()*5
# 		else:
# 			y += y2 + np.random.randn()*5
# 		x_data.append(x)
# 		y_data.append(y)
# np.random.seed ( 0 )
    N = 100
    half_n = N// 2
    r = 10
    s = r /16
    t = np.random.uniform(0 , 2*np.pi , half_n )
    n = s*np.random.randn ( half_n )
    x , y = ( r + n)*np.cos ( t ) , ( r + n)*np.sin ( t )
    X_circ = np.hstack ( ( x.reshape ( half_n , 1 ) , y.reshape ( half_n , 1 ) ) )
    m, b = -1, 2
    x = np.linspace (-12, 12 , half_n )
    y = m*x + b + s*np.random.randn ( half_n )
    X_line = np.hstack ( ( x.reshape ( half_n , 1 ) , y.reshape ( half_n , 1 ) ) )
    X = np.vstack ( ( X_circ , X_line ) )
    xs = [x[0] for x in X]
    ys = [x[1] for x in X]
    return xs,ys 

def circle_func(a, b, r, x):
    return (np.sqrt(r**2-(x-a)**2) + b, -np.sqrt(r**2-(x-a)**2) + b)

class RANSAC:
    def __init__(self, x_data, y_data, n):
        self.x_data = x_data
        self.y_data = y_data
        self.n = n
        self.d_min = 99999
        self.best_model = None

    def random_sampling(self):
        sample = []
        save_ran = []
        count = 0

# get three points from data
        while True:
            ran = np.random.randint(len(self.x_data))

            if ran not in save_ran:
                sample.append((self.x_data[ran], self.y_data[ran]))
                save_ran.append(ran)
                count += 1

                if count == 3:
                    break

        return sample
    
# calculate A, B, C value from three points by using matrix
#pt1 = sample[0]
#pt2 = sample[1]
#pt3 = sample[2]
# A = np.array([[pt2[0] - pt1[0], pt2[1] - pt1[1]], [pt3[0] - pt2[0], pt3[1] - pt2[1]]]) 
# B = np.array([[pt2[0]**2 - pt1[0]**2 + pt2[1]**2 - pt1[1]**2], [pt3[0]**2 - pt2[0]**2 + pt3[1]**2 - pt2[1]**2]])		
# inv_A = inv(A)

# c_x, c_y = np.dot(inv_A, B) / 2
# c_x, c_y = c_x[0], c_y[0]
# r = np.sqrt((c_x - pt1[0])**2 + (c_y - pt1[1])**2)
    def make_model(self, sample):
        cor_1=sample[0]
        cor_2=sample[1]
        cor_3=sample[2]
        x1=cor_1[0]
        y1=cor_1[1]
        x2=cor_2[0]
        y2=cor_2[1]
        x3=cor_3[0]
        y3=cor_3[1]

        c = (x1-x2)**2 + (y1-y2)**2
        a = (x2-x3)**2 + (y2-y3)**2
        b = (x3-x1)**2 + (y3-y1)**2
        s = 2*(a*b + b*c + c*a) - (a*a + b*b + c*c) 
        c_x = (a*(b+c-a)*x1 + b*(c+a-b)*x2 + c*(a+b-c)*x3) / s
        c_y = (a*(b+c-a)*y1 + b*(c+a-b)*y2 + c*(a+b-c)*y3) / s 
        ar = a**0.5
        br = b**0.5
        cr = c**0.5 
        r = ar*br*cr / ((ar+br+cr)*(-ar+br+cr)*(ar-br+cr)*(ar+br-cr))**0.5
        return c_x, c_y, r

    def eval_model(self, model):
        d = 0
        c_x, c_y, r = model

        for i in range(len(self.x_data)):
            dis = np.sqrt((self.x_data[i]-c_x)**2 + (self.y_data[i]-c_y)**2)

            if dis >= r:
                d += dis - r
            else:
                d += r - dis
        return d

    def execute_ransac(self):
        # find best model
        for i in range(self.n):
            model = self.make_model(self.random_sampling())
            d_temp = self.eval_model(model)
            if self.d_min > d_temp:
                self.best_model = model
                self.d_min = d_temp


if __name__ == '__main__':
	# make data
	x_data, y_data = data_generator()

	# show data by scatter type
	plt.scatter(x_data, y_data, c='blue', marker='o', label='data')

	# make ransac class
	# n: how many times try sampling
	ransac = RANSAC(x_data, y_data, 50)
	
	# execute ransac algorithm
	ransac.execute_ransac()
	
	# get best model from ransac
	a, b, r = ransac.best_model[0], ransac.best_model[1], ransac.best_model[2]

	# show result
	circle = plt.Circle((a, b), radius=r, color='r', fc='y', fill=False)
	plt.gca().add_patch(circle)

	plt.axis('scaled')

	# plt.tight_layout()
	plt.show()

In [2]:
import cv2
import numpy as np
import argparse


def click_event(event, x, y, flags, params):

    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(base_image_copy, (x,y), 4, (0,0,255), -1)
        points.append([x, y])
        if len(points) <= 4:
            cv2.imshow('image', base_image_copy)



# Let's sort the points in the following order
# Top-Left, Top-Right, Bottom-Right, Bottom-Left
def sort_pts(points):
    sorted_pts = np.zeros((4, 2), dtype="float32")
    s = np.sum(points, axis=1)
    sorted_pts[0] = points[np.argmin(s)]
    sorted_pts[2] = points[np.argmax(s)]

    diff = np.diff(points, axis=1)
    sorted_pts[1] = points[np.argmin(diff)]
    sorted_pts[3] = points[np.argmax(diff)]

    return sorted_pts



points = []

base_image = cv2.imread(r"D:\01.Vakeesan_vk\01.academics\04. 4 th semester\8 - EN2550 - Fundamentals of image processing\img_process\assignment_02\003.jpg")
base_image_copy = base_image.copy()
subject_image = cv2.imread(r"D:\01.Vakeesan_vk\01.academics\04. 4 th semester\8 - EN2550 - Fundamentals of image processing\img_process\assignment_02\Flag_of_the_United_Kingdom.svg.png")

cv2.imshow('image', base_image_copy)
cv2.setMouseCallback('image', click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()

sorted_pts = sort_pts(points)
h_base, w_base, c_base = base_image.shape
h_subject, w_subject = subject_image.shape[:2]

pts1 = np.float32([[0, 0], [w_subject, 0], [w_subject, h_subject], [0, h_subject]])
pts2 = np.float32(sorted_pts)

# Get the transformation matrix and use it to get the warped image of the subject
transformation_matrix = cv2.getPerspectiveTransform(pts1, pts2)
warped_img = cv2.warpPerspective(subject_image, transformation_matrix, (w_base, h_base))

# Create a mask
mask = np.zeros(base_image.shape, dtype=np.uint8)
roi_corners = np.int32(sorted_pts)

# Fill in the region selected with white color
filled_mask = mask.copy()
cv2.fillConvexPoly(filled_mask, roi_corners, (255, 255, 255))

# Invert the mask color
inverted_mask = cv2.bitwise_not(filled_mask)

# Bitwise AND the mask with the base image
masked_image = cv2.bitwise_and(base_image, inverted_mask)


cv2.imshow('Warped Image', warped_img)
cv2.imshow('Mask Created', mask)
cv2.imshow('Mask after filling', filled_mask)
cv2.imshow('Inverting Mask colors', inverted_mask)
cv2.imshow('Masked Image', masked_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Using Bitwise OR to merge the two images
output = cv2.bitwise_or(warped_img, masked_image)
cv2.imshow('Fused Image', output)
cv2.imwrite('Final_Output.png', output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2 
import matplotlib.pyplot as plt
%matplotlib inline

THRESHOLD = 0.1
NUM_ITERS = 1000

def computeHomography(pairs):
    """
    Solves for the homography given any number of pairs of points. Visit
    http://6.869.csail.mit.edu/fa12/lectures/lecture13ransac/lecture13ransac.pdf
    slide 9 for more details.
    """
    A = []
    for x1, y1, x2, y2 in pairs:
        A.append([x1, y1, 1, 0, 0, 0, -x2 * x1, -x2 * y1, -x1])
        A.append([0, 0, 0, x1, y1, 1, -y2 * x1, -y2 * y1, -y1])
    A = np.array(A)

    # Singular Value Decomposition (SVD)
    U, S, V = np.linalg.svd(A)

    # V has shape (9, 9) for any number of input pairs. V[-1] is the eigenvector
    # of (A^T)A with the smalles eigenvalue. Reshape into 3x3 matrix.
    H = np.reshape(V[-1], (3, 3))

    # Normalization
    H = (1 / H.item(8)) * H
    return H

def dist(pair, H):
    """ Returns the geometric distance between a pair of points given the
    homography H. """
    # points in homogeneous coordinates
    p1 = np.array([pair[0], pair[1], 1])
    p2 = np.array([pair[2], pair[3], 1])

    p2_estimate = np.dot(H, np.transpose(p1))
    p2_estimate = (1 / p2_estimate[2]) * p2_estimate
    
    return np.linalg.norm(np.transpose(p2) - p2_estimate)

def RANSAC(point_map, threshold=THRESHOLD):
    bestInliers = set()
    homography = None
    for i in range(NUM_ITERS):
        # randomly choose 4 points from the matrix to compute the homography
        pairs = [point_map[i] for i in np.random.choice(len(point_map), 4)]

        H = computeHomography(pairs)
        inliers = {(c[0], c[1], c[2], c[3]) for c in point_map if dist(c, H) < 500}
        
      

        if len(inliers) > len(bestInliers):
            bestInliers = inliers
            homography = H
            if len(bestInliers) > (len(point_map) * threshold): break

   

    return homography, bestInliers
# read images
img1 = cv2.imread(r"D:\01.Vakeesan_vk\01.academics\04. 4 th semester\8 - EN2550 - Fundamentals of image processing\img_process\assignment_02\img1.ppm")  
img2 = cv2.imread(r"D:\01.Vakeesan_vk\01.academics\04. 4 th semester\8 - EN2550 - Fundamentals of image processing\img_process\assignment_02\img5.ppm") 

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

#sift
sift = cv2.xfeatures2d.SIFT_create()

keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)

#feature matching
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

matches = bf.match(descriptors_1,descriptors_2)
matches = sorted(matches, key = lambda x:x.distance)


point_map = np.array([[keypoints_1[match.queryIdx].pt[0],
                        keypoints_1[match.queryIdx].pt[1],
                        keypoints_2[match.trainIdx].pt[0],
                        keypoints_2[match.trainIdx].pt[1]] for match in matches])
homography, inliers = RANSAC(point_map)
original_homography=np.array([[6.2544644e-01 ,  5.7759174e-02  , 2.2201217e+02],
                     [ 2.2240536e-01 ,  1.1652147e+00 , -2.5605611e+01],
                     [4.9212545e-04  ,-3.6542424e-05 ,  1.0000000e+00]])
print(original_homography-homography)
dst = cv2.warpPerspective(img2, homography, ((img1.shape[1] + img2.shape[1]), img2.shape[0])) #wraped image

# now paste them together
dst[0:img2.shape[0], 0:img2.shape[1]] = img2
#dst[0:img1.shape[0], 0:img1.shape[1]] = img1
plt.imshow(cv2.cvtColor(dst,cv2.COLOR_BGR2RGB)),plt.show()
plt.imshow(cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)),plt.show()
plt.imshow(cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)),plt.show()
img3 = cv2.drawMatches(img1, keypoints_1, img2, keypoints_2, matches[:50], img2, flags=2)
plt.imshow(img3),plt.show()